# Patient 08 Tumor & Tumor-Blood Matching TCR Analysis

In [ ]:
# load packages
import sys
import scanpy as sc
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.cm as cm


In [ ]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set up figure parameters
plt.rcParams['figure.figsize'] = (3.0, 3.0)
sc.settings.verbosity = 0
sc.set_figure_params(dpi=100, dpi_save=600, format='png', figsize=(3, 3))

In [ ]:
# set working and fig dir
work_dir = "/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/data/TCR_tumor_blood_analysis/Patient_08"
sc.settings.figdir = os.path.join(work_dir, "plots")
fig_dir = os.path.join(work_dir, "plots")

In [ ]:
# read anndata object
adata = sc.read_h5ad("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/data/outputdata/Patient_08/P08_TNK_TCR_updated_07-05-24.h5ad")
adata

In [ ]:
adata.obs

In [ ]:
adata.obs["Annotation_2.0"].unique()

In [ ]:
# basic T cells annotation
sc.pl.umap(
        adata,
        color="cell_type",
        vmin=0,
        vmax="p99",  
        sort_order=False,  
        frameon=False,
        use_raw=False,
        title="Basic T Cells Annotation",
        save = "_P08_Basic_T_Cells_Annotation.png" 
)

In [ ]:
# plot annotation 2.0
sc.pl.umap(
        adata,
        color="Annotation_2.0",
        vmin=0,
        vmax="p99",  
        sort_order=False,  
        frameon=False,
        use_raw=False,
        title="T Cells Annotation",
        save = "_P08_Refined_T_Cells_Annotation.png"
)

In [ ]:
adata.obs['cloneSize'].unique()

In [ ]:
# order cloneSize var
legend_order = ['Hyperexpanded (50 < X <= 100)', 'Large (20 < X <= 50)', 'Medium (5 < X <= 20)',
               'Small (1 < X <= 5)', 'Single (0 < X <= 1)', 'NA']
adata.obs['cloneSize'] = pd.Categorical(adata.obs['cloneSize'], categories=legend_order)
#adata.obs['cloneSize'] = np.array(adata.obs['cloneSize'])

In [ ]:
color_map = cm.get_cmap('inferno')
colorblind_vector = [color_map(i/4) for i in range(4, -1, -1)]
colorblind_vector.append("grey")
print(colorblind_vector)


In [ ]:
# create a dictionary of cloneSize colors
colorblind_dict = dict(map(lambda i,j : (i,j) , legend_order, colorblind_vector))

In [ ]:
def split_umap(adata_list, split_by, titles, ncol=2, nrow=1, **kwargs):
    categories = adata.obs[split_by].cat.categories
    if nrow is None:
        nrow = int(np.ceil(len(categories) / ncol))
    fig, axs = plt.subplots(nrow, ncol, figsize=(4*ncol, 3*nrow)) #5,4
    axs = axs.flatten()
    for i, cat in enumerate(categories):
        ax = axs[i]
        if i != len(categories)-1:
            sc.pl.umap(adata_list[i][adata_list[i].obs[split_by] == cat], title=titles[i],  ax=ax, show=False,  **kwargs)
        else:
            sc.pl.umap(adata_list[i][adata_list[i].obs[split_by] == cat], title=titles[i], ax=ax, show=False,  **kwargs)

    plt.tight_layout()

In [ ]:
# Plot clonal expansion dynamics of tumor clonotypes 
sc.pl.umap(adata, 
           color=["cloneSize"], 
           palette=colorblind_dict,
           na_color = "grey",
           frameon=False,
           title="Tumor Clonotypes",
           legend_fontsize=8,
           save="_P08_Tumor_Clonotypes_Expansion.png"
)

In [ ]:
# Plot clonal expansion dynamics of blood & tumor matching clonotypes 
sc.pl.umap(adata[adata.obs.blood == "TRUE"],  
           color="cloneSize", 
           palette=colorblind_dict,
           frameon=False,
           title="Tumor-blood Clonotypes",
           legend_fontsize=8,
           save="_P08_Tumor-Blood_Clonotypes_Expansion.png"
)

In [ ]:
# order timepoint var
timepoint_order = ['SCR', 'C02']
adata.obs['timepoint'] = pd.Categorical(adata.obs['timepoint'], categories=timepoint_order)

In [ ]:
# Plot clonal expansion dynamics of blood & tumor matching clonotypes split by timepoint
split_umap(adata_list=[adata[adata.obs.blood_pre == "TRUE"], adata[adata.obs.blood_post == "TRUE"]],
           color="cloneSize", 
           split_by="timepoint",
           titles=["T0/-ICI", "T1/+ICI"],
           palette=colorblind_dict,
           frameon=False,
           legend_fontsize=8
          )
plt.savefig(os.path.join(fig_dir, "umap" + "_P08_Tumor-Blood_Clonotypes_Expansion_splitBy_Timepoint.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
split_umap(adata_list=[adata, adata],
           color=["compartment_1013"], 
           split_by="timepoint",
           palette=["grey", "green", "red"],
           frameon=False,
           titles=["T0 - 1013", "T1 - 1013"],
           legend_fontsize=8,
)
plt.savefig(os.path.join(fig_dir, "umap" + "_P08_exp_blood_clonotypes_1013.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
'''
sc.pl.umap(adata[adata.obs.timepoint == "SCR"],  
           color="compartment_1013", 
           palette=colorblind_dict,
           frameon=False,
           legend_fontsize=8
          )
'''

In [ ]:
split_umap(adata_list=[adata, adata],
           color=["compartment_1865"], 
           split_by="timepoint",
           palette=["grey", "green", "red"],
           frameon=False,
           titles=["T0 - 1865", "T1 - 1865"],
           legend_fontsize=8,
           #save="_P08_exp_blood_clonotypes_1013.png"
)
plt.savefig(os.path.join(fig_dir, "umap" + "_P08_exp_blood_clonotypes_1865.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
split_umap(adata_list=[adata, adata],
           color=["compartment_4015"], 
           split_by="timepoint",
           palette=["grey", "green", "red"],
           frameon=False,
           titles=["T0 - 4015", "T1 - 4015"],
           legend_fontsize=8,
           #save="_P08_exp_blood_clonotypes_1013.png"
)
plt.savefig(os.path.join(fig_dir, "umap" + "_P08_exp_blood_clonotypes_4015.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
adata1 = adata[adata.obs['compartment_1013'] != "NA"].copy()
tmp = pd.crosstab(adata1.obs['cell_type'],adata1.obs['compartment_1013'], normalize='columns').T.plot(kind='bar', stacked=True, title="Proportions - 1013")
plt.xlabel(None)
tmp.legend(title='Cell Types', bbox_to_anchor=(1.6, 1.02),loc='upper right')
plt.savefig(os.path.join(fig_dir, "P08_basic_cell_types_props_1013.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
adata2 = adata[adata.obs['compartment_1865'] != "NA"].copy()
tmp = pd.crosstab(adata2.obs['cell_type'],adata2.obs['compartment_1865'], normalize='columns').T.plot(kind='bar', stacked=True, title="Proportions - 1865")
plt.xlabel(None)
tmp.legend(title='Cell Types', bbox_to_anchor=(1.6, 1.02),loc='upper right')
plt.savefig(os.path.join(fig_dir, "P08_basic_cell_types_props_1865.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
adata3 = adata[adata.obs['compartment_4015'] != "NA"].copy()
tmp = pd.crosstab(adata3.obs['cell_type'],adata3.obs['compartment_4015'], normalize='columns').T.plot(kind='bar', stacked=True, title="Proportions - 4015")
plt.xlabel(None)
tmp.legend(title='Cell Types', bbox_to_anchor=(1.6, 1.02),loc='upper right')
plt.savefig(os.path.join(fig_dir, "P08_basic_cell_types_props_4015.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
adata1 = adata[adata.obs['compartment_1013'] != "NA"].copy()
tmp = pd.crosstab(adata1.obs['Annotation_2.0'],adata1.obs['compartment_1013'], normalize='columns').T.plot(kind='bar', stacked=True, title="Proportions - 1013")
plt.xlabel(None)
tmp.legend(title='Cell Types', bbox_to_anchor=(2.2, 1.2),loc='upper right')
plt.savefig(os.path.join(fig_dir, "P08_adv_cell_types_props_1013.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
adata2 = adata[adata.obs['compartment_1865'] != "NA"].copy()
tmp = pd.crosstab(adata2.obs['Annotation_2.0'],adata2.obs['compartment_1865'], normalize='columns').T.plot(kind='bar', stacked=True, title="Proportions - 1865")
plt.xlabel(None)
tmp.legend(title='Cell Types', bbox_to_anchor=(2.2, 1.2),loc='upper right')
plt.savefig(os.path.join(fig_dir, "P08_adv_cell_types_props_1865.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
adata3 = adata[adata.obs['compartment_4015'] != "NA"].copy()
tmp = pd.crosstab(adata3.obs['Annotation_2.0'],adata3.obs['compartment_4015'], normalize='columns').T.plot(kind='bar', stacked=True, title="Proportions - 4015")
plt.xlabel(None)
tmp.legend(title='Cell Types', bbox_to_anchor=(2.2, 1.2),loc='upper right')
plt.savefig(os.path.join(fig_dir, "P08_adv_cell_types_props_4015.png"), dpi=300, format="png", bbox_inches="tight")

## Poster/Thesis Plots


In [ ]:
# load packages
import sys
import scanpy as sc
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.cm as cm

In [ ]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
work_dir = "/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/"
fig_dir = os.path.join(work_dir, "figures", "ISCO24_poster", "Fig6")
sc.settings.figdir = os.path.join(work_dir, "figures", "ISCO24_poster", "Fig6")
sc.set_figure_params(dpi=120, dpi_save=600, format='pdf', frameon=False, figsize=(3,3))

In [ ]:
# read anndata object
adata = sc.read_h5ad("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/data/outputdata/Patient_08/P08_TNK_TCR_updated_07-05-24.h5ad")
adata

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors
tab20c = cm.get_cmap('tab20c')

tab20c_colors = [mcolors.rgb2hex(tab20c(i)) for i in range(tab20c.N)]

index_remove = [16, 17, 19]
index_remove.sort(reverse=True)

for index in index_remove:
    tab20c_colors.pop(index)

for i, color in enumerate(tab20c_colors):
    print(f"Color {i}: {color}")

In [ ]:
adata

In [ ]:
# plot annotation 2.0
sc.pl.umap(
        adata,
        color="Annotation_2.0",
        vmin=0,
        vmax="p99",  
        sort_order=True,  
        frameon=False,
        use_raw=False,
        title="T Cells Annotation (7812)",
        legend_fontsize=8,
        palette=tab20c_colors,
        save = "_P08_TCells_Annotation.pdf"
)

In [ ]:
# plot annotation 2.0
sc.pl.umap(
        adata[adata.obs.timepoint == "SCR"],
        color="Annotation_2.0",
        vmin=0,
        vmax="p99",  
        sort_order=True,  
        frameon=False,
        use_raw=False,
        title="T0/-ICI",
        legend_fontsize=8,
        palette=tab20c_colors,
        save = "_P08_TCells_Annotation_T0.pdf"
)

In [ ]:
# plot annotation 2.0
sc.pl.umap(
        adata[adata.obs.timepoint != "SCR"],
        color="Annotation_2.0",
        vmin=0,
        vmax="p99",  
        sort_order=True,  
        frameon=False,
        use_raw=False,
        title="T1/+ICI",
        legend_fontsize=8,
        palette=tab20c_colors,
        save = "_P08_TCells_Annotation_T1.pdf"
)

In [ ]:
# plot markers
sc.pl.umap(
        adata,
        color=["MKI67", "ZNF683", "GZMB", "HAVCR2", "IL21", "CXCL13", "CXCR5"],
        vmin=0,
        vmax="p99",  
        sort_order=True,  
        frameon=False,
        use_raw=False,
        cmap = "copper"
)

In [ ]:
adata.obs['cloneSize'].unique()

In [ ]:
# order cloneSize var
legend_order = ['Hyperexpanded (50 < X <= 100)', 'Large (20 < X <= 50)', 'Medium (5 < X <= 20)',
               'Small (1 < X <= 5)', 'Single (0 < X <= 1)', 'NA']
adata.obs['cloneSize'] = pd.Categorical(adata.obs['cloneSize'], categories=legend_order)
#adata.obs['cloneSize'] = np.array(adata.obs['cloneSize'])

In [ ]:
color_map = cm.get_cmap('inferno')
colorblind_vector = [color_map(i/4) for i in range(4, -1, -1)]
colorblind_vector.append("grey")
print(colorblind_vector)


In [ ]:
# Plot clonal expansion dynamics of tumor clonotypes 
sc.pl.umap(adata[adata.obs.timepoint == "SCR"], 
           color=["cloneSize"], 
           palette=colorblind_vector,
           na_color = "grey",
           sort_order=True,
           frameon=False,
           use_raw=False,
           title="T0/-ICI",
           legend_fontsize=8,
           save="_P08_Tumor_Clonotypes_Expansion_T0.pdf"
)

In [ ]:
sc.pl.umap(adata[adata.obs.timepoint != "SCR"], 
           color=["cloneSize"], 
           palette=colorblind_vector[1:],
           #na_color = "grey",
           sort_order=True,
           frameon=False,
           use_raw=False,
           title="T1/+ICI",
           legend_fontsize=8,
           save="_P08_Tumor_Clonotypes_Expansion_T1.pdf"
)

In [ ]:
# Plot clonal expansion dynamics of tumor clonotypes 
sc.pl.umap(adata[(adata.obs.timepoint == "SCR") & (adata.obs.blood == "TRUE")], 
           color=["cloneSize"], 
           palette=colorblind_vector,
           na_color = "grey",
           sort_order=True,
           frameon=False,
           use_raw=False,
           title="T0/-ICI",
           legend_fontsize=8,
           #save="_P08_Tumor_Clonotypes_Expansion.png"
)

In [ ]:
# Plot clonal expansion dynamics of tumor clonotypes 
sc.pl.umap(adata[(adata.obs.timepoint != "SCR") & (adata.obs.blood == "TRUE")], 
           color=["cloneSize"], 
           palette=colorblind_vector[1:],
           na_color = "grey",
           sort_order=True,
           frameon=False,
           use_raw=False,
           title="T1/+ICI",
           legend_fontsize=8, 
           #save="_P08_Tumor_Clonotypes_Expansion.png"
)

In [ ]:
# order variable
adata.obs['compartment_4015'] = pd.Categorical(adata.obs['compartment_4015'])
adata.obs['compartment_4015']=adata.obs['compartment_4015'].cat.reorder_categories(['pre_existing', 'de_novo', 'NA'])

In [ ]:
category_mapping = {
    'pre_existing': 'Pre-existing',
    'de_novo': 'De Novo'
}

adata.obs['compartment_4015']=adata.obs['compartment_4015'].cat.rename_categories(category_mapping)

In [ ]:
sc.pl.umap(adata=adata[(adata.obs.blood == "TRUE") & (adata.obs.timepoint == "SCR")],
           color=["compartment_4015"], 
           palette=["forestgreen", "lightgrey"],
           frameon=False,
           title=["T0/-ICI"],
           legend_fontsize=8,
           save="_P08_DiffExp_blood_clonotypes_T0.pdf"
)

In [ ]:
sc.pl.umap(adata=adata[(adata.obs.blood == "TRUE") & (adata.obs.timepoint != "SCR")],
           color=["compartment_4015"], 
           palette=["forestgreen", "orange", "lightgrey"],
           frameon=False,
           title=["T1/+ICI"],
           legend_fontsize=8,
           save="_P08_DiffExp_blood_clonotypes_T1.pdf"
)